In [10]:
import requests
import json
from time import sleep
from datetime import datetime
from socket import gaierror
from tqdm import tqdm

from secrets import API_KEY

In [13]:
def get_bus_lines_stopping(bus_stop_id, bus_stop_nr):
    url = f"https://api.um.warszawa.pl/api/action/dbtimetable_get?id=88cd555f-6f31-43ca-9de4-66c479ad5942&busstopId={bus_stop_id}&busstopNr={bus_stop_nr}&apikey={API_KEY}"

    response = requests.post(url)

    return response

def get_schedule():
    url = f"https://api.um.warszawa.pl/api/action/dbtimetable_get?id=e923fa0e-d96c-43f9-ae6e60518c9f3238&line=wartość&apikey={API_KEY}"

def get_bus_stops():
    url = f"https://api.um.warszawa.pl/api/action/dbstore_get?id=ab75c33d-3a26-4342-b36a-6e5fef0a3ac3&apikey={API_KEY}"

    response = requests.post(url)

    return response

def get_bus_schedule(bus_stop_id, bus_stop_nr, bus_line):
    url = f"https://api.um.warszawa.pl/api/action/dbtimetable_get/?id=e923fa0e-d96c-43f9-ae6e-60518c9f3238&busstopId={bus_stop_id}&busstopNr={bus_stop_nr}&line={bus_line}&apikey={API_KEY}"

    response = None
    while True:
        try:
            response = requests.post(url)
            if response.status_code == 200:
                break
        except requests.exceptions.ConnectionError as e: 
            print(f"connection error, retrying in 3s... {e}")
            sleep(3)
            continue
        except gaierror as e: 
            print(f"connection error, retrying in 3s... {e}")
            sleep(3)
            continue

    return response

In [ ]:
response = get_bus_stops()

data = response.json()

filepath = f"../data/bus-stops.json"
with open(filepath, "w") as json_file:
    json.dump(data, json_file, indent=2)

In [ ]:
bus_stop_ids = [value["values"][0]["value"] for value in data["result"]]
bus_stop_nrs = [value["values"][1]["value"] for value in data["result"]]

bus_stops_iter = zip(bus_stop_ids, bus_stop_nrs)
bus_stop_to_bus_lines_stopping = {}
try:
    for bus_stop_id, bus_stop_nr in tqdm(bus_stops_iter, desc="Downloading data", unit=" iterations", total=len(bus_stop_ids)):
        result = get_bus_lines_stopping(bus_stop_id, bus_stop_nr).json()
        bus_lines_stopping = [kv["values"][0]["value"] for kv in result["result"]]
        bus_stop_to_bus_lines_stopping[(bus_stop_id, bus_stop_nr)] = bus_lines_stopping
except Exception as e:
    print(f"exception {e}")
    print("downloading interrupted")

bus_stop_to_bus_lines_stopping

In [ ]:
data = {f"{id},{nr}": bus_stop_to_bus_lines_stopping[(id, nr)] for id, nr in bus_stop_to_bus_lines_stopping}

data

In [ ]:

filepath = f"../data/bus-stops-to-bus-lines-copy.json"
with open(filepath, "w") as json_file:
    json.dump(data, json_file, indent=2)

In [ ]:
for bus_stop_id, bus_stop_nr in bus_stop_to_bus_lines_stopping:
    bus_lines = bus_stop_to_bus_lines_stopping[(bus_stop_id, bus_stop_nr)]
    for bus_line in bus_lines:
        response = get_bus_schedule(bus_stop_id, bus_stop_nr, bus_line)
        print(response.json())
        break
    break

In [4]:
filepath = "../data/bus-stops-to-bus-lines.json"
with open(filepath, "r", encoding="utf-8") as json_file:
        data = json.load(json_file)

data

{'1001,01': ['102',
  '123',
  '125',
  '138',
  '146',
  '147',
  '166',
  '202',
  '509',
  'N02',
  'N03',
  'N21',
  'N71'],
 '1001,02': ['138', '166', '509', 'N21'],
 '1001,03': ['3', '6', '7', '9', '22', '24', '25', '26'],
 '1001,04': ['1', '3', '4', '6', '20', '25', '26', '28', '73'],
 '1001,05': [],
 '1001,06': ['1', '3', '4', '6', '7', '9', '20', '22', '24', '25', '28', '73'],
 '1001,07': ['120', '123', '169', '509', '512', 'N02', 'N03', 'N71'],
 '1001,08': ['120', '123', '169', '509', '512', 'N02', 'N03', 'N71'],
 '1001,88': ['N02', 'N03', 'N71'],
 '1002,01': ['120', '138', '166', '169', 'N02', 'N03', 'N21', 'N71'],
 '1002,02': ['120', '138', '166', '169', 'N02', 'N03', 'N21', 'N71'],
 '1002,03': ['1', '3', '4', '6', '20', '25', '26', '28', '73'],
 '1002,04': ['1', '3', '4', '6', '20', '25', '26', '28', '73'],
 '1002,06': ['135', '162', '170', '338', 'N14', 'N16', 'N64'],
 '1003,01': ['120',
  '135',
  '162',
  '166',
  '169',
  '170',
  '338',
  '509',
  '512',
  'N02',
  'N

In [14]:
bus_schedules = {}
for key in tqdm(data):
    bus_stop_id, bus_stop_nr = key.split(',')
    for bus_line in data[key]:
        response = get_bus_schedule(bus_stop_id, bus_stop_nr, bus_line)
        stops = response.json()["result"]
        times = [stop["values"][-1]["value"] for stop in stops]
        bus_schedules[f"{bus_stop_id},{bus_stop_nr},{bus_line}"] = times

bus_schedules

  1%|          | 54/8111 [01:31<6:53:48,  3.08s/it]

connection error, retrying in 3s... HTTPSConnectionPool(host='api.um.warszawa.pl', port=443): Max retries exceeded with url: /api/action/dbtimetable_get/?id=e923fa0e-d96c-43f9-ae6e-60518c9f3238&busstopId=1006&busstopNr=04&line=28&apikey=1ec03147-254e-4461-a623-f0e5b7ac2447 (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7f0935d9a5b0>: Failed to establish a new connection: [Errno -3] Temporary failure in name resolution'))
connection error, retrying in 3s... HTTPSConnectionPool(host='api.um.warszawa.pl', port=443): Max retries exceeded with url: /api/action/dbtimetable_get/?id=e923fa0e-d96c-43f9-ae6e-60518c9f3238&busstopId=1006&busstopNr=04&line=28&apikey=1ec03147-254e-4461-a623-f0e5b7ac2447 (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7f0935d9abe0>: Failed to establish a new connection: [Errno -3] Temporary failure in name resolution'))


  2%|▏         | 129/8111 [04:28<4:06:36,  1.85s/it] 

connection error, retrying in 3s... HTTPSConnectionPool(host='api.um.warszawa.pl', port=443): Max retries exceeded with url: /api/action/dbtimetable_get/?id=e923fa0e-d96c-43f9-ae6e-60518c9f3238&busstopId=1025&busstopNr=02&line=900&apikey=1ec03147-254e-4461-a623-f0e5b7ac2447 (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7f0935d9a7c0>: Failed to establish a new connection: [Errno -3] Temporary failure in name resolution'))


  2%|▏         | 133/8111 [04:59<9:23:47,  4.24s/it] 

...

connection error, retrying in 3s... HTTPSConnectionPool(host='api.um.warszawa.pl', port=443): Max retries exceeded with url: /api/action/dbtimetable_get/?id=e923fa0e-d96c-43f9-ae6e-60518c9f3238&busstopId=3325&busstopNr=02&line=119&apikey=1ec03147-254e-4461-a623-f0e5b7ac2447 (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7f093560e7f0>: Failed to establish a new connection: [Errno -3] Temporary failure in name resolution'))


 78%|███████▊  | 6322/8111 [2:29:03<21:20,  1.40it/s]  

connection error, retrying in 3s... HTTPSConnectionPool(host='api.um.warszawa.pl', port=443): Max retries exceeded with url: /api/action/dbtimetable_get/?id=e923fa0e-d96c-43f9-ae6e-60518c9f3238&busstopId=5050&busstopNr=71&line=N45&apikey=1ec03147-254e-4461-a623-f0e5b7ac2447 (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7f093560e370>: Failed to establish a new connection: [Errno -3] Temporary failure in name resolution'))


100%|██████████| 8111/8111 [3:14:28<00:00,  1.44s/it]  


{'1001,01,102': ['05:06:00',
  '05:34:00',
  '05:49:00',
  '06:05:00',
  '06:20:00',
  '06:37:00',
  '06:53:00',
  '07:08:00',
  '07:23:00',
  '07:38:00',
  '07:53:00',
  '08:08:00',
  '08:23:00',
  '08:38:00',
  '08:53:00',
  '09:08:00',
  '09:25:00',
  '09:45:00',
  '10:05:00',
  '10:25:00',
  '10:45:00',
  '11:05:00',
  '11:25:00',
  '11:45:00',
  '12:05:00',
  '12:25:00',
  '12:45:00',
  '13:05:00',
  '13:25:00',
  '13:45:00',
  '14:05:00',
  '14:22:00',
  '14:37:00',
  '14:52:00',
  '15:08:00',
  '15:23:00',
  '15:38:00',
  '15:53:00',
  '16:08:00',
  '16:23:00',
  '16:38:00',
  '16:53:00',
  '17:08:00',
  '17:23:00',
  '17:38:00',
  '17:53:00',
  '18:07:00',
  '18:22:00',
  '18:37:00',
  '18:52:00',
  '19:10:00',
  '19:30:00',
  '19:50:00',
  '20:10:00',
  '20:30:00',
  '20:50:00',
  '21:09:00',
  '21:29:00',
  '21:49:00',
  '22:09:00',
  '22:36:00',
  '23:06:00',
  '23:36:00',
  '24:06:00'],
 '1001,01,123': ['04:56:00',
  '05:26:00',
  '05:52:00',
  '06:08:00',
  '06:24:00',
  '

In [15]:
filepath = f"../data/bus-schedules-copy.json"
with open(filepath, "w") as json_file:
    json.dump(bus_schedules, json_file, indent=2)